In [2]:
import sys
sys.path.append('../')

In [3]:
import AbreivBot.Functions as functions
import AbreivBot.Models.BaseModels as base_model

2023-04-20 00:57:12.648900: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
import pandas as pd
import numpy as np
import string

In [5]:
import tensorflow as tf
devices = tf.config.list_physical_devices()

In [6]:
devices

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [7]:
abbrev = pd.read_csv('TrainAndTest/traindata/cleaned_abbreviations.csv')
abbrev = abbrev[abbrev.columns[1:]]

In [8]:
abbrev['parse_ylabel'] = abbrev.apply(lambda x: ''.join(functions.encode_ylabel(x['Abbreviation'], x['Word'])), axis=1)
abbrev['WordLength'] = abbrev['Word'].apply(lambda x: len(x))

In [9]:
abbrev.head()

,Abbreviation,Word,parse_ylabel,WordLength
0,A.D.,Anno Domini,A.@@@D.@@@@,11
1,A.V.,Authorized Version,A.@@@@@@@@@V.@@@@@,18
2,Abbrev.,abbreviation,Abbrev.@@@@@,12
3,Abbrev.,abbreviations,Abbrev.@@@@@@,13
4,Abd.,Aberdeen,Ab@@d.@@,8


In [10]:
possible_chars = string.ascii_letters + string.digits + ' .@'
possible_chars

'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 .@'

In [11]:
model = base_model.BaseModel(possible_chars)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.build(input_shape=(None, None, len(possible_chars)))

2023-04-20 00:57:15.843193: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-20 00:57:15.845540: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-20 00:57:15.847476: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [12]:
train = abbrev.sample(frac=0.8, random_state=1337)
test = abbrev.drop(train.index)

In [13]:
import AbreivBot.TrainModel as train_model
import AbreivBot.TestModel as test_model

In [14]:
history = train_model.train(model, train, possible_chars, verbose=0)

2023-04-20 00:57:16.746070: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-20 00:57:16.748086: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-20 00:57:16.749712: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [15]:
model.summary()

Model: "base_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  65        
                                                                 
 flatten (Flatten)           multiple                  0 (unused)
                                                                 
 lstm (LSTM)                 multiple                  34060     
                                                                 
 dense (Dense)               multiple                  4290      
                                                                 
Total params: 38,415
Trainable params: 38,415
Non-trainable params: 0
_________________________________________________________________


In [16]:
results = test_model.test(model, test, possible_chars, verbose=0)

2023-04-20 00:58:05.419158: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [32,8,65]
	 [[{{node Placeholder/_1}}]]
2023-04-20 00:58:05.458601: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [21,13,65]
	 [[{{node Placeholder/_1}}]]
2023-04-20 00:58:05.508470: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype float and shape [

In [17]:
results

{8: [2.0040791034698486, 0.49609375],
 13: [2.024660110473633, 0.5604395866394043],
 10: [1.912224292755127, 0.5285714268684387],
 11: [2.457324266433716, 0.4702194333076477],
 14: [2.293689489364624, 0.5158730149269104],
 9: [2.2402374744415283, 0.45679011940956116],
 12: [2.3171041011810303, 0.5080645084381104],
 15: [2.375485420227051, 0.4833333194255829],
 7: [2.268357276916504, 0.4334975481033325],
 16: [3.565251350402832, 0.4375],
 5: [2.7357571125030518, 0.5111111402511597],
 20: [2.7740070819854736, 0.6499999761581421],
 19: [2.2090370655059814, 0.5263158082962036],
 6: [2.151026725769043, 0.49074074625968933],
 4: [2.3017754554748535, 0.625],
 22: [1.8983463048934937, 0.6666666865348816],
 18: [2.781953811645508, 0.3333333432674408],
 17: [1.6900908946990967, 0.529411792755127],
 21: [2.0409419536590576, 0.5714285969734192]}

In [18]:
import time

In [19]:
# model.save_weights('Models/TrainedModels/BaseModel')

In [20]:
ts = int(time.time())
file_path = f"Models/TrainedModels/BaseModel/{ts}/"

In [21]:
model.save(filepath=file_path, save_format='tf')

2023-04-20 00:58:06.601717: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-20 00:58:06.603519: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-20 00:58:06.604848: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

INFO:tensorflow:Assets written to: Models/TrainedModels/BaseModel/1681952286/assets


INFO:tensorflow:Assets written to: Models/TrainedModels/BaseModel/1681952286/assets
